In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [2]:
# Define target ICD-10 prefixes
target_prefixes = ("T40", "F11")

# Path to primary impressions file
primary_path = "../data/raw/FACTPCRPRIMARYIMPRESSION.txt"

# Prepare chunks
primary_chunks = []

with open(primary_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Processing Primary Impressions"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["eSituation_11"] = chunk["eSituation_11"].str.strip(" ~")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        mask = chunk["eSituation_11"].str.startswith(target_prefixes)
        filtered = chunk.loc[mask, ["PcrKey"]]

        if not filtered.empty:
            primary_chunks.append(filtered)

primary_df = pd.concat(primary_chunks, ignore_index=True)
print("Primary impressions matched:", len(primary_df))

Processing Primary Impressions: 530it [00:24, 21.71it/s]

Primary impressions matched: 231354


In [3]:
# Path to secondary impressions file
secondary_path = "../data/raw/FACTPCRSECONDARYIMPRESSION.txt"

# Prepare chunks
secondary_chunks = []

with open(secondary_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Processing Secondary Impressions"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["eSituation_12"] = chunk["eSituation_12"].str.strip(" ~")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        mask = chunk["eSituation_12"].str.startswith(target_prefixes)
        filtered = chunk.loc[mask, ["PcrKey"]]

        if not filtered.empty:
            secondary_chunks.append(filtered)

secondary_df = pd.concat(secondary_chunks, ignore_index=True)
print("Secondary impressions matched:", len(secondary_df))

Processing Secondary Impressions: 544it [00:23, 23.14it/s]

Secondary impressions matched: 42853


In [4]:
# # Path to Primary Symptom
# primary_symptom_path = "../data/raw/FACTPCRPRIMARYSYMPTOM.txt"

# primary_symptom_chunks = []

# with open(primary_symptom_path, "r") as f:
#     for chunk in tqdm(
#         pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
#         desc="Loading Primary Symptom"
#     ):
#         chunk.columns = chunk.columns.str.strip(" ~'")
#         chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")
        
#         filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
#         if not filtered.empty:
#             primary_symptom_chunks.append(filtered)

# primary_symptom_df = pd.concat(primary_symptom_chunks, ignore_index=True)
# print("Primary Symptom records loaded:", len(primary_symptom_df))

# primary_symptom_agg = (
#     primary_symptom_df.groupby("PcrKey")
#     .agg(
#         primary_symptom_first=("eSituation_09", "first"),
#         primary_symptom_count=("eSituation_09", "count"),
#         unique_primary_symptoms=("eSituation_09", "nunique")
#     )
#     .reset_index()
# )


Loading Primary Symptom: 0it [00:00, ?it/s]


NameError: name 'pcr_key_set' is not defined

In [10]:
# # Load SECONDARY impressions
# secondary_uri = "s3://{}/raw-data/FACTPCRSECONDARYIMPRESSION.txt".format(credentials.BUCKET_NAME)
# secondary_chunks = []

# with open(secondary_uri, transport_params=transport_params) as f:
#     for chunk in tqdm(
#         pd.read_csv(f, delimiter="|", chunksize=100_000),
#         desc="Processing Secondary Impressions"
#     ):
#         chunk.columns = chunk.columns.str.strip(" ~'")
#         chunk["eSituation_12"] = chunk["eSituation_12"].astype(str).str.strip(" ~")
#         chunk["PcrKey"] = chunk["PcrKey"].astype(str).str.strip(" ~")

#         mask = chunk["eSituation_12"].str.startswith(target_prefixes)
#         filtered = chunk[mask]

#         if not filtered.empty:
#             secondary_chunks.append(filtered[["PcrKey"]])

# secondary_df = pd.concat(secondary_chunks, ignore_index=True)
# print("Secondary impressions matched:", len(secondary_df))

In [5]:
# Combine primary and secondary
opioid_pcr_df = pd.concat([primary_df, secondary_df]).drop_duplicates()

print("Total unique opioid-related PcrKeys:", len(opioid_pcr_df))

# Convert to set for fast lookup
pcr_key_set = set(opioid_pcr_df["PcrKey"])

Total unique opioid-related PcrKeys: 271206


In [6]:
# Path to Primary Symptom
primary_symptom_path = "../data/raw/FACTPCRPRIMARYSYMPTOM.txt"

primary_symptom_chunks = []

with open(primary_symptom_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Primary Symptom"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")
        
        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            primary_symptom_chunks.append(filtered)

primary_symptom_df = pd.concat(primary_symptom_chunks, ignore_index=True)
print("Primary Symptom records loaded:", len(primary_symptom_df))

primary_symptom_agg = (
    primary_symptom_df.groupby("PcrKey")
    .agg(
        primary_symptom_first=("eSituation_09", "first"),
        primary_symptom_count=("eSituation_09", "count"),
        unique_primary_symptoms=("eSituation_09", "nunique")
    )
    .reset_index()
)


Loading Primary Symptom: 534it [00:33, 15.98it/s]


Primary Symptom records loaded: 267186


In [7]:
cause_injury_path = "../data/raw/FACTPCRCAUSEOFINJURY.txt"

cause_injury_chunks = []

with open(cause_injury_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Cause of Injury"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")
        
        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            cause_injury_chunks.append(filtered)

cause_injury_df = pd.concat(cause_injury_chunks, ignore_index=True)
print("Cause of Injury records loaded:", len(cause_injury_df))

cause_injury_agg = (
    cause_injury_df.groupby("PcrKey")
    .agg(
        cause_injury_first=("eInjury_01", "first"),
        cause_injury_count=("eInjury_01", "count"),
        unique_causes=("eInjury_01", "nunique")
    )
    .reset_index()
)

Loading Cause of Injury: 545it [00:34, 15.57it/s]


Cause of Injury records loaded: 271756


In [12]:
# Path to event file
event_path = "../data/raw/pub_pcrevents_cp25.txt"

# Prepare chunks
event_chunks = []

with open(event_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Event Records"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]

        if not filtered.empty:
            event_chunks.append(filtered)

event_df = pd.concat(event_chunks, ignore_index=True)

print("Event records loaded:", len(event_df))

Loading Event Records: 542it [04:33,  1.98it/s]

Event records loaded: 271206


In [13]:
# Path to Vitals file
vitals_path = "../data/raw/FACTPCRVITAL.txt"

# Define vitals columns to extract and aggregate
vital_cols = {
    "HeartRate": "eVitals_10",
    "RespRate": "eVitals_14",
    "SystolicBP": "eVitals_06",
    "SpO2": "eVitals_12",
    "BGL": "eVitals_18",
    "ETCO2": "eVitals_16",
    "GCS_Eye": "eVitals_19",
    "GCS_Verbal": "eVitals_20",
    "GCS_Motor": "eVitals_21"
}

# Prepare chunks
vitals_chunks = []

with open(vitals_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Vitals"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)].copy()
        if not filtered.empty:
            for name, col in vital_cols.items():
                if col in filtered.columns:
                    extracted = filtered[col].str.extract(r"(\d+\.?\d*)")[0]
                    filtered[name] = pd.to_numeric(extracted, errors="coerce").where(lambda x: x < 1000)
            vitals_chunks.append(filtered[["PcrKey"] + list(vital_cols.keys())])

# Combine all
vitals_df = pd.concat(vitals_chunks, ignore_index=True)

# Aggregate per PcrKey
vitals_agg = (
    vitals_df.groupby("PcrKey")[list(vital_cols.keys())]
    .agg(["first", "last", "min", "max", "mean", "std", "count"])
)

# Flatten column names
vitals_agg.columns = ["_".join(col).strip() for col in vitals_agg.columns.values]
vitals_agg = vitals_agg.reset_index()

print("Vitals aggregated:", vitals_agg.shape)

Loading Vitals: 1648it [05:29,  5.00it/s]


Vitals aggregated: (271071, 64)


In [14]:
# Path to Medications file
meds_path = "../data/raw/FACTPCRMEDICATION.txt"

# Prepare chunks
meds_chunks = []

with open(meds_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Medications"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")
        chunk["eMedications_03Descr"] = chunk["eMedications_03Descr"].str.strip().str.lower()

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            meds_chunks.append(filtered)

# Combine all
meds_df = pd.concat(meds_chunks, ignore_index=True)

print("Medications records loaded:", len(meds_df))

Loading Medications: 628it [01:44,  6.02it/s]

Medications records loaded: 456070


In [15]:
# Flag Naloxone
naloxone_flag = meds_df["eMedications_03Descr"].str.contains("naloxone|narcan", na=False)

# Aggregate medication info
meds_agg = (
    meds_df.groupby("PcrKey")
    .agg(
        total_meds=("eMedications_03", "count"),
        unique_meds=("eMedications_03", "nunique"),
        naloxone_doses=("eMedications_03Descr", lambda x: x.str.contains("naloxone|narcan", na=False).sum()),
        naloxone_flag=("eMedications_03Descr", lambda x: x.str.contains("naloxone|narcan", na=False).any()),
        first_route=("eMedications_07", "first"),
        first_response=("eMedications_10", "first")
    )
    .reset_index()
)

print("Medications aggregated:", meds_agg.shape)

Medications aggregated: (271206, 7)


In [16]:
# Path to Procedures file
proc_path = "../data/raw/FACTPCRPROCEDURE.txt"

# Prepare chunks
proc_chunks = []

with open(proc_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Procedures"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            proc_chunks.append(filtered)

# Combine all
proc_df = pd.concat(proc_chunks, ignore_index=True)

print("Procedures records loaded:", len(proc_df))

Loading Procedures: 934it [01:50,  8.46it/s]

Procedures records loaded: 587755


In [17]:
# Aggregate procedures per PcrKey
proc_agg = (
    proc_df.groupby("PcrKey")
    .agg(
        procedure_count=("eProcedures_03", "count"),
        unique_procedures=("eProcedures_03", "nunique"),
        first_procedure=("eProcedures_03", "first"),
        all_procedures=("eProcedures_03", lambda x: list(x.dropna()))
    )
    .reset_index()
)

# Optionally stringify list for easier storage
proc_agg["all_procedures_str"] = proc_agg["all_procedures"].apply(lambda x: "|".join(x) if x else "")

print("Procedures aggregated:", proc_agg.shape)

Procedures aggregated: (271206, 6)


In [18]:
# Path to CPR records
cpr_path = "../data/raw/FACTPCRARRESTCPRPROVIDED.txt"

# Prepare chunks
cpr_chunks = []

with open(cpr_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading CPR Records"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            cpr_chunks.append(filtered)

cpr_df = pd.concat(cpr_chunks, ignore_index=True)

print("CPR records loaded:", len(cpr_df))

Loading CPR Records: 546it [00:34, 15.81it/s]

CPR records loaded: 275322


In [19]:
# Aggregate CPR info
cpr_agg = (
    cpr_df.groupby("PcrKey")
    .agg(
        cpr_given=("eArrest_09", lambda x: True),
        bystander_cpr=("eArrest_09", lambda x: x.str.contains("BYSTANDER", case=False, na=False).any()),
        ems_cpr=("eArrest_09", lambda x: x.str.contains("EMS|CREW|PROVIDER", case=False, na=False).any())
    )
    .reset_index()
)

print("CPR aggregated:", cpr_agg.shape)

CPR aggregated: (271206, 4)


In [20]:
# Path to ROSC records
rosc_path = "../data/raw/FACTPCRARRESTROSC.txt"

# Prepare chunks
rosc_chunks = []

with open(rosc_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading ROSC Records"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            rosc_chunks.append(filtered)

rosc_df = pd.concat(rosc_chunks, ignore_index=True)

print("ROSC records loaded:", len(rosc_df))

Loading ROSC Records: 543it [00:35, 15.51it/s]

ROSC records loaded: 271712


In [21]:
# Aggregate ROSC flag
rosc_agg = (
    rosc_df.groupby("PcrKey")
    .agg(
        rosc_achieved=("eArrest_12", lambda x: x.str.contains("YES", case=False, na=False).any())
    )
    .reset_index()
)

print("ROSC aggregated:", rosc_agg.shape)

ROSC aggregated: (271206, 2)


In [23]:
# Path to Computed Elements
computed_path = "../data/raw/ComputedElements.txt"

# Prepare chunks
computed_chunks = []

with open(computed_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Computed Demographics"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        # Check which columns exist
        cols_available = [col for col in ["PcrKey", "ePatient_15", "ePatient_16"] if col in chunk.columns]
        if not cols_available:
            continue

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)][cols_available]

        if not filtered.empty:
            computed_chunks.append(filtered)

# Combine
computed_df = pd.concat(computed_chunks, ignore_index=True)

# Deduplicate
demographics_agg = computed_df.drop_duplicates(subset="PcrKey").copy()

# Convert types if columns exist
if "ePatient_15" in demographics_agg.columns:
    demographics_agg["ePatient_15"] = pd.to_numeric(demographics_agg["ePatient_15"], errors="coerce")

if "ePatient_16" in demographics_agg.columns:
    demographics_agg["ePatient_16"] = demographics_agg["ePatient_16"].astype(str).str.strip(" ~")

print("Demographics aggregated:", demographics_agg.shape)

Loading Computed Demographics: 542it [01:13,  7.33it/s]

Demographics aggregated: (271206, 1)


In [24]:
# Path to Alcohol/Drug Use indicators
drug_path = "../data/raw/FACTPCRALCOHOLDRUGUSEINDICATOR.txt"

# Prepare chunks
drug_chunks = []

with open(drug_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Alcohol/Drug Use"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            drug_chunks.append(filtered)

drug_df = pd.concat(drug_chunks, ignore_index=True)

print("Alcohol/Drug Use records loaded:", len(drug_df))

Loading Alcohol/Drug Use: 553it [00:34, 16.03it/s]

Alcohol/Drug Use records loaded: 319541


In [25]:
# Aggregate flags per PcrKey
drug_agg = (
    drug_df.groupby("PcrKey")
    .agg(
        use_flag_count=("eHistory_17", "count"),
        unique_use_flags=("eHistory_17", "nunique"),
        all_use_flags=("eHistory_17", lambda x: list(x.dropna()))
    )
    .reset_index()
)

# Stringify list for storage
drug_agg["all_use_flags_str"] = drug_agg["all_use_flags"].apply(lambda x: "|".join(x) if x else "")

print("Alcohol/Drug Use aggregated:", drug_agg.shape)

Alcohol/Drug Use aggregated: (271206, 5)


In [26]:
# Path to Additional Symptoms
symptom_path = "../data/raw/FACTPCRADDITIONALSYMPTOM.txt"

# Prepare chunks
symptom_chunks = []

with open(symptom_path, "r") as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000, dtype=str),
        desc="Loading Additional Symptoms"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].str.strip(" ~")

        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            symptom_chunks.append(filtered)

symptom_df = pd.concat(symptom_chunks, ignore_index=True)

print("Additional Symptoms records loaded:", len(symptom_df))

Loading Additional Symptoms: 580it [00:34, 16.88it/s]

Additional Symptoms records loaded: 306286


In [27]:
# Aggregate per PcrKey
symptom_agg = (
    symptom_df.groupby("PcrKey")
    .agg(
        symptom_count=("eSituation_10", "count"),
        unique_symptoms=("eSituation_10", "nunique"),
        all_symptoms=("eSituation_10", lambda x: list(x.dropna()))
    )
    .reset_index()
)

# Stringify list for easier storage
symptom_agg["all_symptoms_str"] = symptom_agg["all_symptoms"].apply(lambda x: "|".join(x) if x else "")

print("Additional Symptoms aggregated:", symptom_agg.shape)

Additional Symptoms aggregated: (267904, 5)


In [28]:
# Start with the Event DataFrame (1 row per PcrKey)
df_merged = event_df.copy()

# List of all aggregated DataFrames to merge
feature_tables = {
    "vitals": vitals_agg,
    "medications": meds_agg,
    "procedures": proc_agg,
    "cpr": cpr_agg,
    "rosc": rosc_agg,
    "demographics": demographics_agg,
    "alcohol_drug_use": drug_agg,
    "additional_symptoms": symptom_agg
}

# Merge each
for name, table in feature_tables.items():
    df_merged = df_merged.merge(table, on="PcrKey", how="left")
    print(f"Merged {name}: now shape {df_merged.shape}")

# Save to CSV
df_merged.to_csv("../data/interim/opioid_cases_full.csv", index=False)

print("All data merged and saved to opioid_cases_full.csv")

Merged vitals: now shape (271206, 110)
Merged medications: now shape (271206, 116)
Merged procedures: now shape (271206, 121)
Merged cpr: now shape (271206, 124)
Merged rosc: now shape (271206, 125)
Merged demographics: now shape (271206, 125)
Merged alcohol_drug_use: now shape (271206, 129)
Merged additional_symptoms: now shape (271206, 133)
All data merged and saved to opioid_cases_full.csv
